In [1]:
from bw2data import Database, Method
import bw2data as bd
import brightway2 as bw
import bw2io as bi 
from premise import *

In [2]:
from functions_db import * # this is a python file with self-created functions needed to run this notebook
from config import * #some main key parameters needed.
from private_keys import * #import premise key

In [3]:
# Decide to generate the new activities in the databases, this must be done once before going to the next Jupyter notebook 
GENERATE_ACTS = False

In [4]:
PROJECT_NAME

'iea_h2_lca'

In [5]:
bw.projects.set_current(PROJECT_NAME) #Creating/accessing the project
DB_NAME_INIT,PROJECT_NAME

('ecoinvent-3.10-cutoff', 'iea_h2_lca')

In [6]:
all_ei_databases = [db_2 for db_2 in bw.databases if "ecoinvent_" in str(db_2)]
#all_ei_databases = all_ei_databases[3:] + all_ei_databases[:3]
all_ei_databases

['ecoinvent_310_reference',
 'ecoinvent_remind_SSP2-PkBudg1150_2030_base',
 'ecoinvent_remind_SSP2-PkBudg1150_2035_base',
 'ecoinvent_remind_SSP2-PkBudg1150_2040_base',
 'ecoinvent_image_SSP2-RCP26_2030_base',
 'ecoinvent_image_SSP2-RCP26_2035_base',
 'ecoinvent_image_SSP2-RCP26_2040_base']

### 0. Import ecoinvent database and generate reference premise db for the current year importing additional inventories

In [7]:
# Create dbs if not existing yet
if DB_NAME_INIT not in bw.databases:
    bi.import_ecoinvent_release("3.10", "cutoff", USER_NAME, USER_PW)
if NAME_REF_DB not in bw.databases:
    clear_cache()
    generate_reference_database()

### 1. Make a new LCIA category for climate change including impacts for emissions to air from hydrogen

In [8]:
flows = []

if ('EF v3.1 EN15804', 'climate change', 'global warming potential (GWP100) including H2') not in bd.methods:
    # Start from the IPCC 2021 method
    for flow in bw.Method(('EF v3.1 EN15804', 'climate change', 'global warming potential (GWP100)')).load():
        flows.append(flow)
    
    # finally, add hydrogen flow
    
    for bio in bw.Database(BIOSPHERE_DB):
        if "Hydrogen" == bio['name'] and 'air' in str(bio['categories']) and bio['unit'] == "kilogram":
            bio_key = bio[1]
            print(bio)
            flows.append(((BIOSPHERE_DB, bio_key), 11)) # use a GWP of 11 for H2, according to premise-gwp and other refs.
    
    # write flows
    my_method = Method(('EF v3.1 EN15804', 'climate change', 'global warming potential (GWP100) including H2'))
    my_metadata = {"unit": "kg CO2-eq.", "meaning": "to represent climate change"}
    my_method.register(**my_metadata)
        
    bw.Method(('EF v3.1 EN15804', 'climate change', 'global warming potential (GWP100) including H2')).write(flows)

### 2. We need to create additional prospective LCA databases since many of the IEA projects are for the near-future up to 2040, we use a 2 degree C

In [9]:
list_spec_scenarios, list_names = generate_future_ei_dbs(scenarios = ["SSP2-PkBudg1150"], iam = 'remind',
                                                         start_yr=2025, end_yr = 2040, step = 5, endstring="base")
list_spec_scenarios, list_names

Avoid duplicated db and therefore following db not added: 'ecoinvent_remind_SSP2-PkBudg1150_2030_base'
Avoid duplicated db and therefore following db not added: 'ecoinvent_remind_SSP2-PkBudg1150_2035_base'
Avoid duplicated db and therefore following db not added: 'ecoinvent_remind_SSP2-PkBudg1150_2040_base'


([], [])

In [10]:
if len(list_spec_scenarios)>0:
    generate_prospective_lca_dbs(list_spec_scenarios, list_names)

In [11]:
list_spec_scenarios, list_names = generate_future_ei_dbs(scenarios = ['SSP2-RCP26'], iam = 'image',
                                                         start_yr=2025, end_yr = 2040, step = 5, endstring="base")
list_spec_scenarios, list_names

Avoid duplicated db and therefore following db not added: 'ecoinvent_image_SSP2-RCP26_2030_base'
Avoid duplicated db and therefore following db not added: 'ecoinvent_image_SSP2-RCP26_2035_base'
Avoid duplicated db and therefore following db not added: 'ecoinvent_image_SSP2-RCP26_2040_base'


([], [])

In [12]:
if len(list_spec_scenarios)>0:
    generate_prospective_lca_dbs(list_spec_scenarios, list_names)

In [13]:
all_ei_databases = [db_2 for db_2 in bw.databases if "ecoinvent_" in str(db_2)]
#all_ei_databases = all_ei_databases[3:] + all_ei_databases[:3]
all_ei_databases

['ecoinvent_310_reference',
 'ecoinvent_remind_SSP2-PkBudg1150_2030_base',
 'ecoinvent_remind_SSP2-PkBudg1150_2035_base',
 'ecoinvent_remind_SSP2-PkBudg1150_2040_base',
 'ecoinvent_image_SSP2-RCP26_2030_base',
 'ecoinvent_image_SSP2-RCP26_2035_base',
 'ecoinvent_image_SSP2-RCP26_2040_base']

change_petro = True

for db in all_ei_databases[1:]:
    act_ffs = [act for act in bd.Database(db) if 'fossil fuel' == act['name'] and 
                 'fossil fuel' == act['name'] ]

    # add separately an activity representing direct reduced iron inclusion
    for act_ff in act_ffs:
        for exc in act_ff.exchanges():
            if exc['name'] == 'market for petroleum' and 'petroleum' == exc['product']:
                init_amount = exc['amount']
                print(exc['name'], exc['location'],init_amount)
                if change_petro:
                    # This is the only activity with sponge iron, aware that this is for India though (the market only consists of India for DRI).
                    print(f"Manually change petro exc {exc['name']} [{exc['location']}]")
                    add_act = find_bw_act(db, 'market for petroleum','petroleum', 'RoW')
                    exc.delete()
            
                    # Add the exchange
                    act_ff.new_exchange(input=add_act.key, amount=init_amount, 
                                         name = add_act['name'],
                                         product = add_act['reference product'],
                                    location = add_act['location'], unit=add_act['unit'],type='technosphere').save()
                    exc.save()
                    act_ff.save()

### 2.1, Next, we will now have to generate additional activities, which account for burning methanol, synfuels, and methane

In [13]:
kg_co2_uptake = 3.12 # NEste
kg_co2_combustion_ethanol = 1.91 #kgCO2/kg fuel
kg_co2_combustion_methanol = 1.37 #kgCO2/kg fuel
kg_co2_combustion_methanol_ms = 1.6904231625835189 #kgCO2/kg fuel, this was included in the fuel itself source from the atmosphere, so we use the same as emission to balance the carbon balance.
kg_co2_combustion_diesel_jf = 3.16 #kgCO2/kg fuel for both jet and diesel

acts_check = [ # Biofuels:
                ('biofuel, Neste NExBTL', 'biofuel, Neste NExBTL', 'GLO'),
              ('diesel production, petroleum refinery operation', 'diesel', 'RoW'),
            ('ethanol production, via fermentation, from corn, economic allocation', 'ethanol, from corn', 'US'),
                #Synfuels:
                ('kerosene production, synthetic, Fischer Tropsch process, hydrogen from electrolysis, energy allocation', 'kerosene, synthetic', 
                 'RER', 'EUR'),
                ('kerosene production, petroleum refinery operation', 'kerosene', 'Europe without Switzerland'),
                ('kerosene production, synthetic, Fischer Tropsch process, hydrogen from wood gasification, energy allocation', 
                 'kerosene, synthetic', 'RER'),
                # Methanol:
                ('methanol synthesis, hydrogen from electrolysis, CO2 from DAC', 'methanol, unpurified', 'RER', 'EUR'),
              ('market for methanol', 'methanol', 'RoW'),
                ('methanol production, biomass gasification', 'methanol, from biomass', 'RoW'),
             ]

if GENERATE_ACTS:
    for act_info in acts_check[3:]:
        print(act_info[1])
        for db in all_ei_databases:
            if ('reference' in str(db)) and (act_info[2] == 'EUR'):
                # those REMIND locations are not in the ref db
                print(f"No activity found for name '{act_info[0]}' in location '{act_info[2]}' in database: {db}'")
                continue                
            else:
                if (len(act_info) > 3) and ('reference' not in str(db)):
                    loc_select = act_info[3] # take the prospective location name
                    if 'image' in str(db) and loc_select == 'EUR':
                        loc_select = 'WEU'
                        print(f"Used 'WEU' instead of 'EUR' for {db} in Europe")
                else:
                    loc_select = act_info[2] # take the current location name     
                    
                # find bw act we want to copy and modify
                act_init = find_bw_act(db, act_info[0], act_info[1], loc_select)
        
                new_name = f"{act_init['name']} incl. combustion"
                check_act = [act for act in bw.Database(db) if new_name == act['name'] and loc_select == act['location']]
        
                #copy and make new one
                if len(check_act) == 0:
                    act = act_init.copy()
                else:
                    print("WARNING: Activity '{}'[{}] already generated, deleting!".format(new_name,loc_select))
                    act_delete = find_bw_act(db, new_name, act_info[1], loc_select)
                    act_delete.delete()
                    act = act_init.copy()
                
                act['name'] = new_name
                act.save()
                #biofuels, so the CO2 flow is different
                if ('biofuel' in str(act_info[0]) ):
                    #print(act['reference product'], act['name'])
        
                    # add CO2 uptake
                    bio_key = search_biosphere_entries(BIOSPHERE_DB, 'Carbon dioxide, in air', ('natural resource', 'in air'), 'kilogram').key
                    act.new_exchange(input=bio_key,amount=kg_co2_uptake, unit="kilogram",type='biosphere').save()
        
                    # add bio CO2 emission
                    # Carbon dioxide, from soil or biomass stock	('air',)	1.0	kilogram
                    bio_key = search_biosphere_entries(BIOSPHERE_DB, 'Carbon dioxide, from soil or biomass stock', 
                                                       ('air',), 'kilogram').key
                    
                    act.new_exchange(input=bio_key,amount=kg_co2_uptake, unit="kilogram",type='biosphere').save()
        
                elif ('from corn' in str(act_info[0]) ):
                    # add bio CO2 emission for ethanol from corn
                    # Carbon dioxide, from soil or biomass stock	('air',)	1.0	kilogram
                    bio_key = search_biosphere_entries(BIOSPHERE_DB, 'Carbon dioxide, from soil or biomass stock', 
                                                       ('air',), 'kilogram').key
                    
                    act.new_exchange(input=bio_key,amount=kg_co2_combustion_ethanol, unit="kilogram",type='biosphere').save()
                    
                # add fossil CO2 emission
                elif ('kerosene' in str(act_info[0]) ) or ('diesel production' in str(act_info[0]) ):
                    #synfuels, with fossil CO2 flow
                    # Carbon dioxide, fossil	('air',)	1.0	kilogram
                    
                    bio_key = search_biosphere_entries(BIOSPHERE_DB, 'Carbon dioxide, fossil' if 'wood gasification' not in str(act_info[0]) else 'Carbon dioxide, from soil or biomass stock', 
                                                       ('air',), 'kilogram').key
                    
                    act.new_exchange(input=bio_key,amount=kg_co2_combustion_diesel_jf, unit="kilogram",type='biosphere').save()
        
                # add fossil CO2 emission
                elif 'methanol synthesis, hydrogen from electrolysis, CO2 from DAC' in str(act_info[0]):
                    #synfuels
                    # Carbon dioxide, non-fossil	('air',)	1.0
                    bio_key = search_biosphere_entries(BIOSPHERE_DB, 'Carbon dioxide, non-fossil', 
                                                       ('air',), 'kilogram').key
                    
                    act.new_exchange(input=bio_key,amount=kg_co2_combustion_methanol_ms, unit="kilogram",type='biosphere').save()
             
                else:
                    #methanol
                    # Carbon dioxide, from soil or biomass stock	('air',)	1.0	kilogram
                    bio_key = search_biosphere_entries(BIOSPHERE_DB, 'Carbon dioxide, fossil' if 'biomass gasification' not in str(act_info[0]) else 'Carbon dioxide, from soil or biomass stock', 
                                                       ('air',), 'kilogram').key
                    
                    act.new_exchange(input=bio_key,amount=kg_co2_combustion_methanol, unit="kilogram",type='biosphere').save()
                        

In [14]:
conv_factor = 0.735 #kgCH4/m3
burning_kg_ch4 = 2.74 #kgCO2/kgCH4
kg_co2_combustion_m3_ng = burning_kg_ch4 * conv_factor#kgCO2/m3 volume

acts_check = [ # Methane
                ('methane, from electrochemical methanation, with carbon from atmosphere',	'methane, from electrochemical methanation',	'RER'),
              ('market for natural gas, high pressure',	'natural gas, high pressure',	'RoW'),
            ('market for biomethane, high pressure', 'biomethane, high pressure', 'RoW'),
             ]

if GENERATE_ACTS:
    for act_info in acts_check:
        print(act_info[1])
        for db in all_ei_databases:
            if ('reference' in str(db)) and (act_info[2] == 'EUR'):
                # those REMIND locations are not in the ref db
                print(f"No activity found for name '{act_info[0]}' in location '{act_info[2]}' in database: {db}'")
                continue
            else:
                # find bw act we want to copy and modify
                act_init = find_bw_act(db, act_info[0], act_info[1], act_info[2])
        
                new_name = f"{act_init['name']} incl. combustion"
                check_act = [act for act in bw.Database(db) if new_name == act['name'] and act_info[2] == act['location']]
        
                #copy and make new one
                if len(check_act) == 0:
                    act = act_init.copy()
                else:
                    print("WARNING: Activity '{}'[{}] already generated, deleting!".format(new_name,act_info[2]))
                    act_delete = find_bw_act(db, new_name, act_info[1], act_info[2])
                    act_delete.delete()
                    act = act_init.copy()
                
                act['name'] = new_name
                act.save()
                #biofuels, so the CO2 flow is different
                if ('biomethane' in str(act_info[0]) ):
                    #print(act['reference product'], act['name'])
        
                    # add bio CO2 emission
                    # Carbon dioxide, from soil or biomass stock	('air',)	1.0	kilogram
                    bio_key = search_biosphere_entries(BIOSPHERE_DB, 'Carbon dioxide, from soil or biomass stock', 
                                                       ('air',), 'kilogram').key
                    
                    act.new_exchange(input=bio_key,amount=kg_co2_combustion_m3_ng, unit="kilogram",type='biosphere').save()
                    
                # add fossil CO2 emission
                elif ('market for natural gas' in str(act_info[0]) ):
                    #synfuels, with fossil CO2 flow
                    # Carbon dioxide, fossil	('air',)	1.0	kilogram
                    bio_key = search_biosphere_entries(BIOSPHERE_DB, 'Carbon dioxide, fossil', 
                                                       ('air',), 'kilogram').key
                    act.new_exchange(input=bio_key,amount=kg_co2_combustion_m3_ng, unit="kilogram",type='biosphere').save()
        
                # add fossil CO2 emission
                elif ('carbon from atmosphere' in str(act_info[0]) ):
                    #synfuels
                    # Carbon dioxide, non-fossil	('air',)	1.0
                    bio_key = search_biosphere_entries(BIOSPHERE_DB, 'Carbon dioxide, non-fossil', 
                                                       ('air',), 'kilogram').key
                    
                    act.new_exchange(input=bio_key,amount=burning_kg_ch4, unit="kilogram",type='biosphere').save()
             

### 2.2, For steel production, since premise changes the foreground of steel production, we should also (partly) do so for our new LCIs:

In [15]:
act_infos = [
            ('H2-DRI, iron production',	
            'H2-DRI iron', 'GLO'),
            ('H2-DRI-EAF, steel production, low-alloyed',	
            'H2-DRI-EAF steel, low-alloyed', 'GLO'),]

improvement_steel =  { 'ecoinvent_310_reference':0,
	'ecoinvent_remind_SSP2-PkBudg1150_2030_base': 0.04,
	'ecoinvent_remind_SSP2-PkBudg1150_2035_base': 0.12,
	'ecoinvent_remind_SSP2-PkBudg1150_2040_base': 0.22,
    'ecoinvent_image_SSP2-RCP26_2030_base': 0.16,
	'ecoinvent_image_SSP2-RCP26_2035_base': 0.16,
	'ecoinvent_image_SSP2-RCP26_2040_base': 0.16,                     
                     } #those values are taken from primary steel production in premise.

if GENERATE_ACTS:
    for i, db in enumerate(all_ei_databases):
        print(db)
        for act_info in act_infos:
            act = find_bw_act(db, act_info[0], act_info[1], act_info[2])
    
            new_name =f'{act_info[0]}, foreground changed'
            check_act = [act for act in bw.Database(db) if new_name == act['name'] and act_info[2] == act['location']]
    
            #copy and make new one
            if len(check_act) == 0:
                act = act.copy()
            else:
                print("WARNING: Activity '{}'[{}] already generated, deleting!".format(new_name,act_info[2]))
                act_delete = find_bw_act(db, new_name, act_info[1], act_info[2])
                act_delete.delete()
                act = act.copy()
    
            act['name'] = new_name
            act.save()
            
            for exc in act.exchanges():
                factor = 1-(improvement_steel[db])
                if 'electricity' in str(exc['name']):
                    print(f"Changed foreground with factor {factor} for {act['name']}:{exc['name']} from {round(exc['amount'],3)} to {round(exc['amount']*factor,3)}")
                    exc['amount'] = exc['amount']*factor
                    exc.save()
                elif 'natural gas' in str(exc['name']):
                    print(f"Changed foreground with factor {factor} for {act['name']}:{exc['name']} from {round(exc['amount'],3)} to {round(exc['amount']*factor,3)}")
                    exc['amount'] = exc['amount']*factor 
                    exc.save()
                elif ('Carbon dioxide, fossil' in str(exc['name']) ) and (exc['type']=='biosphere'): 
                    print(f"Changed foreground with factor {factor} for {act['name']}:{exc['name']} from {round(exc['amount'],3)} to {round(exc['amount']*factor,3)}")
                    exc['amount'] = exc['amount']*factor 
                    exc.save()
                    
            for exc in act.exchanges():            
                if (exc['name'] == 'H2-DRI, iron production') and (exc['type']=='technosphere'):
                    exc.delete()
                    add_act = find_bw_act(db, 'H2-DRI, iron production, foreground changed', 'H2-DRI iron', 'GLO')
                    act.new_exchange(amount= 1.061992490719463, product = add_act['reference product'], location = add_act['location'],
                                     name = add_act['name'], input = add_act.key, type="technosphere").save()
                    print(f"Old H2-DRI exchange removed and successfully linked to new H2-DRI exchanges for act: {act_info[0]}.")
            act.save()

In [16]:
act_infos = [ ('BF+CCS, iron production',	 'BF+CCS iron', 'GLO'),
            ('BF-BOF+CCS, steel production, low-alloyed', 'steel, low-alloyed', 'GLO'),]

if GENERATE_ACTS:
    for i, db in enumerate(all_ei_databases):
        print(db)
        for act_info in act_infos:
            act = find_bw_act(db, act_info[0], act_info[1], act_info[2])
    
            new_name =f'{act_info[0]}, foreground changed'
            check_act = [act for act in bw.Database(db) if new_name == act['name'] and act_info[2] == act['location']]
    
            #copy and make new one
            if len(check_act) == 0:
                act = act.copy()
            else:
                print("WARNING: Activity '{}'[{}] already generated, deleting!".format(new_name,act_info[2]))
                act_delete = find_bw_act(db, new_name, act_info[1], act_info[2])
                act_delete.delete()
                act = act.copy()
    
            act['name'] = new_name
            act.save()
            
            for exc in act.exchanges():
                factor = 1-(improvement_steel[db])
                if 'electricity' in str(exc['name']):
                    print(f"Changed foreground with factor {factor} for {act['name']}:{exc['name']} from {round(exc['amount'],3)} to {round(exc['amount']*factor,3)}")
                    exc['amount'] = exc['amount']*factor
                    exc.save()
                elif 'natural gas' in str(exc['name']):
                    print(f"Changed foreground with factor {factor} for {act['name']}:{exc['name']} from {round(exc['amount'],3)} to {round(exc['amount']*factor,3)}")
                    exc['amount'] = exc['amount']*factor 
                    exc.save()
                elif ('Carbon dioxide, fossil' in str(exc['name']) ) and (exc['type']=='biosphere'): 
                    print(f"Changed foreground with factor {factor} for {act['name']}:{exc['name']} from {round(exc['amount'],3)} to {round(exc['amount']*factor,3)}")
                    exc['amount'] = exc['amount']*factor 
                    exc.save()
                    
            for exc in act.exchanges():            
                if (exc['name'] == 'BF+CCS, iron production') and (exc['type']=='technosphere'):
                    exc.delete()
                    add_act = find_bw_act(db, 'BF+CCS, iron production, foreground changed', 'BF+CCS iron', 'GLO')
                    act.new_exchange(amount= 1.066809403, product = add_act['reference product'], location = add_act['location'],
                                     name = add_act['name'], input = add_act.key, type="technosphere").save()
                    print(f"Old BF+CCS iron exchange removed and successfully linked to new BF+CCS iron exchanges for act: {act_info[0]}.")
            act.save()

### 2.3, To make sure that we everywhere use primary steel, make a new activity to make sure we only have primary steel in 'steel production, converter, low-alloyed'

In [17]:
act_info = ('steel production, converter, low-alloyed',	
            'steel, low-alloyed',	'RER')

amount_primary_steel = 0.198864791 + 0.867944612 # market for iron scrap, sorted, pressed + market for pig iron  IN KILOGRAM 

if GENERATE_ACTS:
    for i, db in enumerate(all_ei_databases):
        print(db)
        if ('reference' in str(db)) and (act_info[2] == 'EUR'):
            # those REMIND locations are not in the ref db
            print(f"No activity found for name '{act_info[0]}' in location '{act_info[2]}' in database: {db}'")
            continue
        else:
            loc_spec = "EUR" if 'reference' not in str(db) else act_info[2]
            if 'image' in str(db) and loc_spec == 'EUR':
                loc_spec = 'WEU'
                print(f"Used 'WEU' instead of 'EUR' for {db} in Europe")
                
            # find bw act we want to copy and modify
            act_init = find_bw_act(db, act_info[0], act_info[1], loc_spec)
        
            new_name ='steel production, converter, low-alloyed, only primary steel'
            check_act = [act for act in bw.Database(db) if new_name == act['name'] and act_info[2] == act['location']]
        
            #copy and make new one
            if len(check_act) == 0:
                act = act_init.copy()
            else:
                print("WARNING: Activity '{}'[{}] already generated, deleting!".format(new_name,act_info[2]))
                act_delete = find_bw_act(db, new_name, act_info[1], act_info[2])
                act_delete.delete()
                act = act_init.copy()
        
            act['name'] = new_name
            act['location'] = 'RER'
            act.save()
    
            # Get the amount of CO2 directly emitted, as proxy for the factor used that is captured
            for exc in act.exchanges():
                if ('market for pig iron' == exc['name']) and (exc['type']=='technosphere'):
                    print(f"Changed amount_primary_steel to {amount_primary_steel} for {exc['name']}")
                    exc['amount'] = amount_primary_steel
                    exc.save()
                if ('market for iron scrap, sorted, pressed' == exc['name']) and (exc['type']=='technosphere'):
                    print(f"Changed scrap steel amount to 0 for {exc['name']}")
                    exc['amount'] = 0
                    exc.save()
    
            act.save()

act_infos = [('steel production, converter, low-alloyed, only primary steel',	
            'steel, low-alloyed',	'RER')]

# Also, change foreground, as done with other steel activities
if GENERATE_ACTS:
    for i, db in enumerate(all_ei_databases):
        print(db)
        for act_info in act_infos:
            act = find_bw_act(db, act_info[0], act_info[1], act_info[2])
    
            new_name =f'{act_info[0]}, foreground changed'
            check_act = [act for act in bw.Database(db) if new_name == act['name'] and act_info[2] == act['location']]
    
            #copy and make new one
            if len(check_act) == 0:
                act = act.copy()
            else:
                print("WARNING: Activity '{}'[{}] already generated, deleting!".format(new_name,act_info[2]))
                act_delete = find_bw_act(db, new_name, act_info[1], act_info[2])
                act_delete.delete()
                act = act.copy()
    
            act['name'] = new_name
            act.save()
            
            for exc in act.exchanges():
                factor = 1-(improvement_steel[db])
                if 'electricity' in str(exc['name']):
                    print(f"Changed foreground with factor {factor} for {act['name']}:{exc['name']} from {round(exc['amount'],3)} to {round(exc['amount']*factor,3)}")
                    exc['amount'] = exc['amount']*factor
                    exc.save()
                elif 'natural gas' in str(exc['name']):
                    print(f"Changed foreground with factor {factor} for {act['name']}:{exc['name']} from {round(exc['amount'],3)} to {round(exc['amount']*factor,3)}")
                    exc['amount'] = exc['amount']*factor 
                    exc.save()
                elif ('Carbon dioxide, fossil' in str(exc['name']) ) and (exc['type']=='biosphere'): 
                    print(f"Changed foreground with factor {factor} for {act['name']}:{exc['name']} from {round(exc['amount'],3)} to {round(exc['amount']*factor,3)}")
                    exc['amount'] = exc['amount']*factor 
                    exc.save()    
            act.save()

### 2.4. Now, generate additional hydrogen production activities (based on water electrolysis) with different CFs and power sources considering three different electroyzers, credits for the LCIs developed in premise (Sacchi et al., 2022).

In [18]:
def create_new_activity_with_new_exc(act_full, new_act_to_add, NAME_REF_DB, BIOSPHERE_DB, target_db, add_str=""):
    """
    Create a new activity in a target database by copying and modifying an existing activity,
    then adding exchanges from a target activity to the new activity.
    
    Parameters:
    - act_full (list): A list containing details of the existing activity to be copied and modified.
      It should include:
        * The full activity name (e.g., "transport, freight, lorry, battery electric, NMC-622 battery, 32t gross weight, long haul").
        * The reference product (e.g., "transport, freight, lorry").
        * The location (e.g., "RER").
    - new_act_to_add (list): A list containing details of the new activity to add:
        * The new activity name (e.g., "market for electricity, low voltage").
        * The new reference product (e.g., "electricity, low voltage").
        * The new location (e.g., "NL").
    - NAME_REF_DB (object): The database object for retrieving and referencing activities by name.
    - BIOSPHERE_DB (object): The database object for biosphere exchanges.
    - target_db (object): The database object where the new activity will be created.
    
    Returns:
    - act_new (object): The newly created activity object.
    """
    
    exchanges = []

    name_search = f"{act_full[0]}{add_str}" if len(add_str)>0 else act_full[0]

    # check whether hydrogen production facility already exists.
    matching_activities = [
        act for act in bw.Database(target_db)
        if name_search == act['name'] and new_act_to_add[2] == act['location']
        and act_full[1] == act['reference product'] ]
    
    # if already exists, give warning, otherwise create a new activity referring to a hydrogen production facility
    if len(matching_activities) > 0:
        #print("WARNING: Activity already generated: {} [{}]".format(name_act_facility,loc))
        print(f"Activity already generated for '{name_search}' and '{new_act_to_add}'")  

    else:
        # Find and modify the existing activity
        act_sel = find_bw_act(NAME_REF_DB, act_full[0], act_full[1], act_full[2])
    
        for exc in act_sel.exchanges(): 
            if exc['type'] == 'technosphere':
                if new_act_to_add[1] == exc['product']:
                    bw_act_to_add = find_bw_act(NAME_REF_DB, new_act_to_add[0], new_act_to_add[1], new_act_to_add[2])
                    exchanges.append({
                        'name': new_act_to_add[0],
                        'reference product': new_act_to_add[1],
                        'location': new_act_to_add[2], 
                        'input': bw_act_to_add.key,
                        'amount': exc['amount'],
                        'type': 'technosphere',
                    })
                else:
                    exchanges.append(exc.as_dict())
            if exc['type'] == 'biosphere':
                exchanges.append(exc.as_dict())
    
        # Create new activity
        new_act_code = str(uuid.uuid4().hex)
        act_new = bw.Database(target_db).new_activity(
            **{
                'activity': act_full[0],
                'activity type': 'production',
                'code': new_act_code,
                'location': new_act_to_add[2],
                'name': f"{act_full[0]}{add_str}" if len(add_str)>0 else act_full[0],
                'production amount': 1,
                'reference product': act_full[1],
                'unit': act_sel['unit'],
                'comment': ""
            }
        )
    
        # Add production exchange
        act_new.new_exchange(**{
            'input': (target_db, new_act_code),
            'amount': 1,
            'type': 'production',
            'formula': ""
        }).save()
    
        # Add exchanges from the selected activity to the new activity
        for exc in exchanges:
            try:
                exc['reference product'] = exc['product']
            except:
                pass 
                    
            if exc['type'] == 'technosphere':
                add_exchange_to_activity(act_new, exc['name'], exc['location'], 
                                         exc['reference product'], exc['amount'], target_db)
            if exc['type'] == 'biosphere':
                add_exchange_to_activity_bio(act_new, exc['name'], exc['categories'], exc['amount'], BIOSPHERE_DB) 
    
        # Save the new activity
        act_new.save()

### 2.5 Now, generate additional hydrogen production activities (based on water electrolysis) with different CFs and power sources considering three different electroyzers, credits for the LCIs developed in premise (Sacchi et al., 2022).
NOTE: This takes long as we write to the existing databases

#### Generate new activities for electrolytic hydrogen production needed as sources in our LCIs.

In [19]:
all_ei_databases

['ecoinvent_310_reference',
 'ecoinvent_remind_SSP2-PkBudg1150_2030_base',
 'ecoinvent_remind_SSP2-PkBudg1150_2035_base',
 'ecoinvent_remind_SSP2-PkBudg1150_2040_base',
 'ecoinvent_image_SSP2-RCP26_2030_base',
 'ecoinvent_image_SSP2-RCP26_2035_base',
 'ecoinvent_image_SSP2-RCP26_2040_base']

for db in all_ei_databases[:2]:
    print(db)
    for act in bd.Database(db):
        if ', global cf' in str(act['name']):
            print(act['name'])
            act.delete()
        if 'electrolysis, power from' in str(act['name']) and 'hydrogen production, gaseous' in str(act['name']):
            print(act['name'])
            act.delete()

In [20]:
GENERATE_ACTS = True

In [21]:
all_ei_databases = [db_2 for db_2 in bw.databases if "ecoinvent_image_SSP2-RCP26" in str(db_2) or
                   "ecoinvent_remind_SSP2-PkBudg1150_2040_base" in str(db_2)
                   
                   ]
#all_ei_databases = all_ei_databases[3:] + all_ei_databases[:3]
all_ei_databases

['ecoinvent_remind_SSP2-PkBudg1150_2040_base',
 'ecoinvent_image_SSP2-RCP26_2030_base',
 'ecoinvent_image_SSP2-RCP26_2035_base',
 'ecoinvent_image_SSP2-RCP26_2040_base']

In [22]:
cf_hydro = 0.8 # IEA, excel sheet of hydrogen production facilities
cf_grid = 0.57 # IEA, excel sheet of hydrogen production facilities
cf_nuclear = 0.8 # IEA, excel sheet of hydrogen production facilities

if GENERATE_ACTS: 
    #loop over generated dbs of premise
    for db in all_ei_databases:
        print(db)
        for electr in ["pem", "aec", "soec"]:
            print(electr)
            list_cfs_solar = [round(x,3) for x in np.arange(0.10, 0.30, 0.025)]
            list_cfs_wind = [round(x,2) for x in np.arange(0.2, 0.75, 0.05)]
            
            for x in list_cfs_solar:
                curtail_pv = curve_fit_f(x, db, "ratio_pv_curtailed")
                # generate h2 production activities for ground-mounted solar PV
                GENERATE_ACTS_GM_PV(db, x, curtail_pv, cf_electrolyzer=0.3, electrolyzer = electr,
                                   excel_col_name = db)
                
            for x in list_cfs_wind:
                #generate activities for onshore and offshore wind
                curtail_wind_on = curve_fit_f(x, db, "ratio_wind_curtailed_on")
                GENERATE_ACTS_WIND(db, x, curtail_wind_on, cf_electrolyzer=0.4, electrolyzer = electr,
                                   excel_col_name = db)
            
            curtail_wind_off = 0
            GENERATE_ACTS_WIND_OFF(db, 0.55, curtail_wind_off, cf_electrolyzer=0.55, electrolyzer = electr,
                               excel_col_name = db)
        
            # Make hydropower based H2 production activities for water electrolysis
            # Here, we assume pumper hydro is this is the largest growing hydropower source, see:
            # https://iea.blob.core.windows.net/assets/4d2d4365-08c6-4171-9ea2-8549fabd1c8d/HydropowerSpecialMarketReport_corr.pdf
            act_hydro = ["electricity production, hydro, reservoir, alpine region", 'electricity, high voltage']
            
            all_db_locs_hy = [act['location'] for act in bw.Database(db) 
                               if act_hydro[0] == act['name'] and act_hydro[1] == act['reference product'] 
                           and len(act['location'])<=3]   # to only get countries and not provinces 
    
            # Loop over locations and generate them
            for loc in all_db_locs_hy:
                GENERATE_ACTS_GENERIC_ELECT(db, cf_electrolyzer=cf_hydro, electrolyzer = electr,
                                                generic_electr_act = [act_hydro[0],
                                                                      act_hydro[1],
                                                                      loc], excel_col_name = db)
                                                
            # Make power grid based H2 production activities for water electrolysis        
            act_grid = ['market for electricity, medium voltage', 'electricity, medium voltage']
            
            all_db_locs_grid = [act['location'] for act in bw.Database(db) 
                               if act_grid[0] == act['name'] and act_grid[1] == act['reference product'] 
                           and len(act['location'])<=3]  # to only get countries and not provinces 
            
            for loc in all_db_locs_grid:
                GENERATE_ACTS_GENERIC_ELECT(db, cf_electrolyzer=cf_grid, electrolyzer = electr,
                                                generic_electr_act = [act_grid[0],
                                                                      act_grid[1],
                                                                      loc], excel_col_name = db)
    
            # For aggregated market group power zones for grid-based water electrolysis
            act_grid = ['market group for electricity, medium voltage', 'electricity, medium voltage'] 
            
            all_db_locs_grid = [act['location'] for act in bw.Database(db) 
                               if act_grid[0] == act['name'] and act_grid[1] == act['reference product'] 
                           and len(act['location'])<=3]  # to only get countries and not provinces 
            
            for loc in all_db_locs_grid:
                GENERATE_ACTS_GENERIC_ELECT(db, cf_electrolyzer=cf_grid, electrolyzer = electr,
                                                generic_electr_act = [act_grid[0],
                                                                      act_grid[1],
                                                                      loc], excel_col_name = db)
    
            # For water electrolysis using power from nuclear power plants
            # larges from PWR: https://world-nuclear.org/nuclear-essentials/are-there-different-types-of-reactor, assume that here.
            act_nuclear = ['electricity production, nuclear, pressure water reactor', 'electricity, high voltage'] 
            
            all_db_locs_nucl = [act['location'] for act in bw.Database(db) 
                               if act_nuclear[0] == act['name'] and act_nuclear[1] == act['reference product'] 
                           and len(act['location'])<=3]  # to only get countries and not provinces 
            
            for loc in all_db_locs_nucl:#[:3]:
                GENERATE_ACTS_GENERIC_ELECT(db, cf_electrolyzer=cf_nuclear, electrolyzer = electr,
                                                generic_electr_act = [act_nuclear[0],
                                                                      act_nuclear[1],
                                                                      loc], excel_col_name = db)

        """"""
        """This is for generating low-carbon truck activities using solar PV"""
        """"""
        act_low_carbon_power = "electricity production, photovoltaic, commercial"
        act_low_carbon_power_ref = "electricity, low voltage"
        act_low_carbon_power_loc = "ES" # Spain as a proxy with relative high solar PV potential


        # Define the activity details to be copied and modified
        act_full = [
            "transport, freight, lorry, battery electric, 32t gross weight, long haul",  # Full activity name
            "transport, freight, lorry",  # Reference product
            "RER"  # Location
        ]
        
        # Define the new activity details to be added
        new_act_to_add_large = [
            act_low_carbon_power,  # New activity name
            act_low_carbon_power_ref,  # New reference product
            act_low_carbon_power_loc,
        ]
        
        create_new_activity_with_new_exc(
                            act_full, #reference act to copy and modify
                            [act_low_carbon_power, act_low_carbon_power_ref, act_low_carbon_power_loc], #new exchange to add, replacing ref product new_act_to_add[1]
                            db,
                            BIOSPHERE_DB,
                            db,
                            add_str = ", solar-based power"
                        )

        """"""
        """This is for generating low-carbon BEV activities using solar PV"""
        """"""
        
        # Define the activity details to be copied and modified
        act_full = [
            "transport, passenger car, battery electric, Medium",  # Full activity name
            "transport, passenger car",  # Reference product
            "RER"  # Location
        ]
        
        create_new_activity_with_new_exc(
                            act_full, #reference act to copy and modify
                            [act_low_carbon_power, act_low_carbon_power_ref, act_low_carbon_power_loc], #new exchange to add, replacing ref product new_act_to_add[1]
                            db,
                            BIOSPHERE_DB,
                            db,
                            add_str = ", solar-based power"
                        )

        """This is for generating low-carbon heat pump activity using solar PV"""
        """"""

        # Define the activity details to be copied and modified
        act_full = [
            "heat production, air-water heat pump 10kW",  # Full activity name
            "heat, air-water heat pump 10kW",  # Reference product
            "RoW"  # Location
        ]
        
        create_new_activity_with_new_exc(
                            act_full, #reference act to copy and modify
                            [act_low_carbon_power, act_low_carbon_power_ref, act_low_carbon_power_loc], #new exchange to add, replacing ref product new_act_to_add[1]
                            db,
                            BIOSPHERE_DB,
                            db,
                            add_str = ", solar-based power"
                        )

ecoinvent_remind_SSP2-PkBudg1150_2040_base
pem
aec
soec
ecoinvent_image_SSP2-RCP26_2030_base
pem
Skipped: activity 'hydrogen production, gaseous, 30 bar, from PEM electrolysis, power from market for electricity, medium voltage' already generated!
aec
Skipped: activity 'hydrogen production, gaseous, 30 bar, from AEC electrolysis, power from market for electricity, medium voltage' already generated!
soec
Skipped: activity 'hydrogen production, gaseous, 30 bar, from SOEC electrolysis, power from market for electricity, medium voltage' already generated!
ecoinvent_image_SSP2-RCP26_2035_base
pem
Skipped: activity 'hydrogen production, gaseous, 30 bar, from PEM electrolysis, power from market for electricity, medium voltage' already generated!
aec
Skipped: activity 'hydrogen production, gaseous, 30 bar, from AEC electrolysis, power from market for electricity, medium voltage' already generated!
soec
Skipped: activity 'hydrogen production, gaseous, 30 bar, from SOEC electrolysis, power from m